In [1]:
import os

os.chdir("../..")
print(os.getcwd())

/data1/xhuan192/codes/med-sipf


In [2]:
import dotenv
dotenv.load_dotenv(override=True)

True

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import json
from typing import List, Union
from pprint import pprint, pformat
import seaborn as sns
import datasets
import tqdm
from datetime import datetime

/data1/xhuan192/misc/miniconda3/envs/m1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from hashlib import sha256
def get_str_hash(input_string: str) -> str:
    return sha256(input_string.encode()).hexdigest()[:16]

In [5]:
repo_id = "mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong-decon_eval"

split = "train"
dataset = datasets.load_dataset(repo_id, split=split)
def add_prompt_hash(sample):
    return {"prompt_hash": get_str_hash(sample["prompt"])}
dataset = dataset.map(add_prompt_hash)

In [6]:
domain_repo_id = "mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong-decon_eval-tokenized-120325-extract-domain-code"
split = "train"
prompt_domain_dataset = datasets.load_dataset(domain_repo_id, split=split)
prompt_domain_dataset = prompt_domain_dataset.map(add_prompt_hash)
prompt_domain_mapping = prompt_domain_dataset.to_pandas().set_index("prompt_hash").to_dict(orient="index")

def add_domain(sample):
    return prompt_domain_mapping[sample["prompt_hash"]]

dataset = dataset.map(add_domain)

In [7]:
dataset[0]

{'answer_idx': 2,
 'source': 'GBaker/MedQA-USMLE-4-options',
 'metadata': '{\'question\': "A mother brings her 3-week-old infant to the pediatrician\'s office because she is concerned about his feeding habits. He was born without complications and has not had any medical problems up until this time. However, for the past 4 days, he has been fussy, is regurgitating all of his feeds, and his vomit is yellow in color. On physical exam, the child\'s abdomen is minimally distended but no other abnormalities are appreciated. Which of the following embryologic errors could account for this presentation?", \'answer\': \'Abnormal migration of ventral pancreatic bud\', \'options\': {\'A\': \'Abnormal migration of ventral pancreatic bud\', \'B\': \'Complete failure of proximal duodenum to recanalize\', \'C\': \'Abnormal hypertrophy of the pylorus\', \'D\': \'Failure of lateral body folds to move ventrally and fuse in the midline\'}, \'meta_info\': \'step1\', \'answer_idx\': \'A\', \'metamap_phras

In [8]:
# Domain-source sampling
domain_source_samples = {}
for sample in tqdm.tqdm(dataset):
    domain_code = sample["domain_code"]
    if domain_code not in domain_source_samples:
        domain_source_samples[domain_code] = {}
    
    current_domain = domain_source_samples[domain_code]
    source_name = "merge_source"
    if source_name not in current_domain:
        current_domain[source_name] = []

    current_domain[source_name].append(sample)


100%|██████████| 23493/23493 [00:03<00:00, 6159.12it/s]


In [9]:
def get_num_samples_from_domain(domain_source_samples):
    num_samples = 0
    for domain_code in domain_source_samples:
        domain = domain_source_samples[domain_code]
        print(domain_code)
        for source_name in domain:
            print(f"  {source_name}: {len(domain[source_name])}")
            num_samples += len(domain[source_name])
    return num_samples

num_samples = get_num_samples_from_domain(domain_source_samples)
if num_samples != len(dataset):
    raise ValueError(f"Number of samples do not match, {num_samples} != {len(dataset)}")

27
  merge_source: 258
54
  merge_source: 1189
20
  merge_source: 1928
50
  merge_source: 1602
32
  merge_source: 796
65
  merge_source: 757
53
  merge_source: 1231
38
  merge_source: 168
42
  merge_source: 178
02
  merge_source: 1056
01
  merge_source: 602
08
  merge_source: 80
19
  merge_source: 306
44
  merge_source: 385
49
  merge_source: 81
24
  merge_source: 679
45
  merge_source: 33
37
  merge_source: 443
16
  merge_source: 701
17
  merge_source: 441
33
  merge_source: 364
06
  merge_source: 2348
00
  merge_source: 671
23
  merge_source: 119
07
  merge_source: 129
55
  merge_source: 188
30
  merge_source: 30
09
  merge_source: 406
36
  merge_source: 218
34
  merge_source: 257
52
  merge_source: 347
56
  merge_source: 149
18
  merge_source: 94
67
  merge_source: 427
29
  merge_source: 276
66
  merge_source: 236
63
  merge_source: 61
21
  merge_source: 751
48
  merge_source: 215
13
  merge_source: 99
75
  merge_source: 107
10
  merge_source: 247
57
  merge_source: 207
51
  merge_s

In [10]:
seed = 42
rng = np.random.default_rng(42)

target_num_samples = 1000
selected_samples = []

pbar = tqdm.tqdm(total=target_num_samples, desc="Sampling")
while len(selected_samples) < target_num_samples:
    # sample a domain
    domain_code = rng.choice(list(domain_source_samples.keys()))

    chosen_domain = domain_source_samples[domain_code]
    # remove empty domain
    if len(chosen_domain) == 0:
        pbar.write(f"Before sampling sources, removing domain {domain_code}")
        domain_source_samples.pop(domain_code)
        continue
    # sample a source
    source_name = rng.choice(list(chosen_domain.keys()))
    chosen_domain_source = chosen_domain[source_name]

    if not isinstance(chosen_domain_source, list):
        raise ValueError(f"Expected list, got {type(chosen_domain_source)}")

    # remove empty domain source
    if len(chosen_domain_source) == 0:
        pbar.write(f"Before sampling samples, removing domain {domain_code} source {source_name}")
        chosen_domain.pop(source_name)
        continue

    # choose a sample, and remove the sample from the chosen_domain_source
    before_rm_length = len(chosen_domain_source)
    sample = rng.choice(chosen_domain_source)
    chosen_domain_source.remove(sample)
    after_rm_length = len(chosen_domain_source)
    if before_rm_length != after_rm_length + 1:
        raise ValueError(f"Expected {before_rm_length} == {after_rm_length} + 1")

    # double check remove empty domain source
    if len(chosen_domain_source) == 0:
        pbar.write(f"Removing source {source_name} from domain {domain_code}")
        chosen_domain.pop(source_name)
    
    # double check remove empty domain
    if len(chosen_domain) == 0:
        pbar.write(f"Removing domain {domain_code}")
        domain_source_samples.pop(domain_code)

    selected_samples.append(sample)
    pbar.update(1)


Sampling:  81%|████████▏ | 814/1000 [00:00<00:00, 5029.11it/s]

Removing source merge_source from domain 71
Removing domain 71
Removing source merge_source from domain 31
Removing domain 31
Removing source merge_source from domain 74
Removing domain 74
Removing source merge_source from domain 12
Removing domain 12
Removing source merge_source from domain 40
Removing domain 40
Removing source merge_source from domain 46
Removing domain 46
Removing source merge_source from domain 04
Removing domain 04
Removing source merge_source from domain 72
Removing domain 72
Removing source merge_source from domain 60
Removing domain 60
Removing source merge_source from domain 64
Removing domain 64


In [11]:
print(f"Total samples: {len(dataset)}")
print(f"Selected {len(selected_samples)} samples")
print(f"Remaining samples: {get_num_samples_from_domain(domain_source_samples)}")

Total samples: 23493
Selected 1000 samples
27
  merge_source: 242
54
  merge_source: 1175
20
  merge_source: 1912
50
  merge_source: 1584
32
  merge_source: 781
65
  merge_source: 745
53
  merge_source: 1211
38
  merge_source: 148
42
  merge_source: 160
02
  merge_source: 1041
01
  merge_source: 582
08
  merge_source: 71
19
  merge_source: 287
44
  merge_source: 373
49
  merge_source: 72
24
  merge_source: 672
45
  merge_source: 23
37
  merge_source: 423
16
  merge_source: 688
17
  merge_source: 433
33
  merge_source: 352
06
  merge_source: 2328
00
  merge_source: 657
23
  merge_source: 97
07
  merge_source: 118
55
  merge_source: 177
30
  merge_source: 21
09
  merge_source: 392
36
  merge_source: 204
34
  merge_source: 244
52
  merge_source: 329
56
  merge_source: 134
18
  merge_source: 81
67
  merge_source: 415
29
  merge_source: 261
66
  merge_source: 221
63
  merge_source: 48
21
  merge_source: 737
48
  merge_source: 197
13
  merge_source: 85
75
  merge_source: 96
10
  merge_source

In [12]:
selected_dataset = datasets.Dataset.from_list(selected_samples)

In [13]:
push_repo_id = f"{repo_id}-domain_1k"
selected_dataset.push_to_hub(push_repo_id)


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 15.58ba/s]


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.13it/s]
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong-decon_eval-domain_1k/commit/878ee4fb97bf68d10e1fc339f21e7918ee2d448f', commit_message='Upload dataset', commit_description='', oid='878ee4fb97bf68d10e1fc339f21e7918ee2d448f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong-decon_eval-domain_1k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong-decon_eval-domain_1k'), pr_revision=None, pr_num=None)

In [14]:
push_repo_id

'mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong-decon_eval-domain_1k'

In [ ]:
from collections import Counter

def stat_domain_dataset(dataset, path):
    source_list = dataset["source"]
    source_counter = Counter(source_list)
    source_counter = {k: v for k, v in source_counter.items()}

    domain_counter = Counter(dataset["domain_name"])
    domain_counter = {k: v for k, v in domain_counter.items()}

    num_samples = len(dataset)
    source_counter_perecent = {k: v / num_samples for k, v in source_counter.items()}

    print(f"{path}\n{dataset}\n{pformat(source_counter)}\n{pformat(source_counter_perecent)}\n{pformat(domain_counter)}\n")

stat_domain_dataset(selected_dataset, push_repo_id)

mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong-decon_eval-domain_1k
Dataset({
    features: ['answer_idx', 'source', 'metadata', 'prompt', 'answer_letter', 'answer_string', 'qwen_7b-extracted_answer_string', 'qwen_7b-model_answer_string', 'qwen_7b-correct', 'qwen_32b-extracted_answer_string', 'qwen_32b-model_answer_string', 'qwen_32b-correct', 'reasoning', 'distilled_answer_string', 'r1-extracted_answer_string', 'r1-model_answer_string', 'r1-correct', 'prompt_hash', 'domain', 'domain_code', 'domain_name', 'is_domain_code_extracted'],
    num_rows: 1000
})
{'GBaker/MedQA-USMLE-4-options': 52,
 'openlifescienceai/headqa': 20,
 'openlifescienceai/medmcqa': 924,
 'qiaojin/PubMedQA:pqa_labeled': 4}
{'GBaker/MedQA-USMLE-4-options': 0.052,
 'openlifescienceai/headqa': 0.02,
 'openlifescienceai/medmcqa': 0.924,
 'qiaojin/PubMedQA:pqa_labeled': 0.004}
{'Abnormalities': 14,
 'Administration & Dosage': 20,
 'Adverse Effects': 15,
 'Agonists': 14,
 'Analogs & Derivatives': 11,
 'Analysis': 10,

Sampling: 100%|██████████| 1000/1000 [00:11<00:00, 5029.11it/s]